In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sqlalchemy import create_engine, text
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely import wkt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from sqlalchemy import create_engine, text
import geopy.distance

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
liqour = gpd.read_file('../data/liquor_licenses.zip')

In [17]:
liqour

,BFN,BUS_PROF_N,FULL_ADDRE,LICENSES,LIC_STATUS,ISSUE_DATE,END_DATE,ADDRESS_ID,ADDRESS_LI,ADDRESS__1,CITY,STATE,ZIP,COUNCIL_DI,POLICE_DIS,CENSUS_TRA,NEIGHBORHO,ZONE_DISTR,X_COORD,Y_COORD,HEARING_DA,HEARING_TI,HEARING_ST,HEARING__1,geometry,month,month_name,year
71,2001-BFN-77661,ROCKET LOUNGE,None,LIQUOR - TAVERN AND CABARET,CLOSED - EXPIRED,2015-09-28,2013-01-19,0.0,None,None,None,None,None,0,None,None,None,None,10000.0,0.0,None,None,None,None,POINT (-115.43111 34.64123),9,September,2015
103,2023-AMEND-0000109,COSTCO WHOLESALE#1652,None,COMBINED LIQUOR - TRANSFER LOCATION,CLOSED - ADMINISTRATIVELY,2023-01-27,None,0.0,None,None,None,None,None,0,None,None,None,None,10000.0,0.0,None,None,None,None,POINT (-115.43111 34.64123),1,January,2023
136,2001-BFN-5787,PAGLIACCIS,1440 W 33RD AVE,LIQUOR - HOTEL AND RESTAURANT,CLOSED - EXPIRED,2015-08-31,2012-12-31,0.0,None,None,None,None,80211,0,None,None,None,None,0.0,0.0,None,None,None,None,POINT (-115.46406 34.63823),8,August,2015
331,2001-BFN-4861,BOB & JOHN'S CAROUSEL LOUNGE,None,LIQUOR - TAVERN,CLOSED - EXPIRED,2018-07-19,2011-12-14,0.0,None,None,None,None,None,0,None,None,None,None,10000.0,0.0,None,None,None,None,POINT (-115.43111 34.64123),7,July,2018
494,2001-BFN-62519,RESTAURANT JAPON,None,LIQUOR - HOTEL/RESTAURANT,CLOSED - EXPIRED,2014-11-06,2014-11-05,0.0,None,None,None,None,None,0,None,None,None,None,10000.0,0.0,None,None,None,None,POINT (-115.43111 34.64123),11,November,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9100,2008-BFN-1037986,LOUNGE 5280 LLC CONCOURSE B,8600 PENA BLVD,LIQUOR - HOTEL AND RESTAURANT,LICENSE ISSUED - ACTIVE,2024-05-10,2025-04-26,221158.0,8600 Pena Blvd,None,Denver,CO,80249,11,7,08313,DIA,DIA,3231946.0,1736179.0,None,None,None,None,POINT (-104.67389 39.85173),5,May,2024
9101,2008-BFN-1032241,STAYBRIDGE SUITES DIA,6951 N TOWER,LIQUOR - TAVERN,DELINQUENT,2024-06-17,2024-06-17,163317.0,6951 N Tower Rd,None,Denver,CO,None,11,5,08313,DIA,C-MU-20,3204198.0,1725553.0,None,None,None,None,POINT (-104.77302 39.82321),6,June,2024
9102,2012-BFN-1059860,MY GOODS MARKET #6546,7680 PENA BLVD,LIQUOR - FERMENTED MALT BEVERAGE,LICENSE ISSUED - ACTIVE,2023-09-12,2024-09-22,223533.0,7680 Pena Blvd,None,Denver,CO,80249,11,7,08313,DIA,DIA,3219006.0,1730351.0,None,None,None,None,POINT (-104.72016 39.83604),9,September,2023
9103,2008-BFN-1034524,BOULDER BEER TAP HOUSE,8400 PENA T5-26,LIQUOR - HOTEL AND RESTAURANT,LICENSE ISSUED - ACTIVE,2024-03-19,2024-11-09,32749249.0,8400 Pena Blvd Bldg T5 # 26,None,Denver,CO,0,11,7,08313,DIA,DIA,3231993.0,1735200.0,None,None,None,None,POINT (-104.67376 39.84904),3,March,2024


In [12]:
liqour['LICENSES'].value_counts()

LICENSES
LIQUOR - HOTEL AND RESTAURANT                                       1175
LIQUOR - HOTEL AND RESTAURANT AND CABARET                            377
LIQUOR - STORE                                                       301
LIQUOR - TAVERN                                                      272
LIQUOR - TAVERN AND CABARET                                          269
LIQUOR - BEER & WINE                                                 252
LIQUOR - FERMENTED MALT BEVERAGE                                     242
LIQUOR - HOTEL/RESTAURANT                                            156
LIQUOR - ART GALLERY PERMIT                                          147
LIQUOR - 3.2% BEER                                                    58
LIQUOR - LODGING AND ENTERTAINMENT                                    52
LIQUOR - 3.2 % BEER                                                   51
LIQUOR - RETAIL                                                       33
LIQUOR - CLUB                             

In [9]:
liqour = liqour[liqour['LICENSES'] != 'SPECIAL EVENTS']

In [11]:
liqour = liqour[~liqour['LICENSES'].isin(['LIQUOR - SPECIAL EVENTS',
                                          'LIQUOR - TEMPORARY 120 DAYS',
                                          'LIQUOR - CHANGE OF OWNER',
                                         'LIQUOR - SPECIAL EVENT 3.2%- BUS, RCPT,NP-PER DAY',
                                         'LIQUOR - LATE RENEWAL PENALTY',
                                         'LIQUOR - TASTINGS'])]

In [16]:
liqour['ISSUE_DATE'] = pd.to_datetime(liqour['ISSUE_DATE'], format='ISO8601')
liqour['month'] = liqour['ISSUE_DATE'].dt.month
liqour['month_name'] = liqour['ISSUE_DATE'].dt.month_name()
liqour['year'] = liqour['ISSUE_DATE'].dt.year

In [18]:
liqour.to_csv('../data/liqour_clean.csv', index=False)